In [25]:
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

from siriuspy.search import IDSearch, PSSearch, MASearch
from siriuspy.devices import VPU, SOFB, HLFOFB, IDFF, Tune, DVFImgProc, PowerSupply, BunchbyBunch
from siriuspy.devices.bbb import Acquisition, SingleBunch
from siriuspy.magnet.factory import NormalizerFactory
from siriuspy.devices import DVF, DVFImgProc

import epics
from mathphys.functions import save, load

# Functions definition

In [36]:
def initialize_vpu(beamline):
    # Search ID
    devnamevpu = IDSearch.conv_beamline_2_idname(beamline=beamline)
    vpu = VPU(devname=devnamevpu)

    # Disable beamline control
    vpu.cmd_beamline_ctrl_disable()
    print('beamline control: ', vpu.is_beamline_ctrl_enabled)

    return vpu

def move_vpu_gap(vpu:VPU, gap, timeout, verbose=False):
    vpu.set_gap(gap)
    time.sleep(0.5)
    print('Gap-RB {:.3f} mm'.format(vpu.gap)) if verbose else 0
    if vpu.cmd_move_start(timeout):
        time.sleep(0.5)
        print('Undulator is moving...') if verbose else 0
        while vpu.is_moving:
            time.sleep(0.1)
            print('Current gap {:.3f} mm.'.format(vpu.gap_mon), end='\r') if verbose else 0
        print('Gap {:.3f} mm reached.'.format(vpu.gap)) if verbose else 0
        return True
    else:
        print('Error while cmd_move_start.')
        return False

def move_vpu_gap_robust(vpu:VPU, gap, timeout, maxiter=3, verbose=False):
    sucess = move_vpu_gap(vpu, gap=gap, timeout=timeout, verbose=verbose)
    i=0
    while not sucess and i<maxiter:
        i += 1
        print('Trying {:0f}/{:.0f}'.format(i, maxiter)) if verbose else 0
        vpu.cmd_reset(timeout=timeout)
        time.sleep(0.5)
        sucess = move_vpu_gap(vpu, gap=gap, timeout=timeout, verbose=verbose)
    if sucess:
        print('Movimentation done!\n')
        return True
    else:
        print('Error while moving.\n')
        return False

def get_tune_data(bbb_h, bbb_v,  tune, nr_acq=20, acq_sleep=1):
    tunex = np.zeros(nr_acq)
    tuney = np.zeros(nr_acq)
    tunex_dev = np.zeros(nr_acq)
    tuney_dev = np.zeros(nr_acq)
    for i in range(nr_acq):
        tunex[i] = bbb_h.spec_marker1_tune
        tuney[i] = bbb_v.spec_marker1_tune
        tunex_dev[i] = tune.tunex
        tuney_dev[i] = tune.tuney
        time.sleep(acq_sleep)
    return tunex, tuney, tunex_dev, tuney_dev



# Initialize devices

In [27]:
timeout = 3

In [29]:
vpu = initialize_vpu(beamline='CARNAUBA')
tune = Tune(Tune.DEVICES.SI)
tune.wait_for_connection(timeout=timeout)

sofb = SOFB(SOFB.DEVICES.SI)
fofb = HLFOFB(HLFOFB.DEVICES.SI)

beamline control:  False
center mode:  0
pitch mode:  0
gap speed: 0.500 mm/s
gap: 24.000 mm
beamline control:  False
center mode:  0
pitch mode:  0
gap speed: 0.500 mm/s
gap: 24.000 mm


# Turn on autocorr

In [30]:
sofb.cmd_turn_on_autocorr(timeout=timeout)
fofb.cmd_turn_on_loop_state(timeout=timeout)

True

# Create bbb

In [31]:
bbb_v = SingleBunch(BunchbyBunch.DEVICES.V)
bbb_v.spec_marker1_tune

bbb_h = SingleBunch(BunchbyBunch.DEVICES.H)
bbb_h.spec_marker1_tune

tune = Tune(Tune.DEVICES.SI)

# Meas ID effect on tunes

In [32]:
gaps = np.linspace(24, 4, 21)
gaps[-1] = 4.3
gaps

array([24. , 23. , 22. , 21. , 20. , 19. , 18. , 17. , 16. , 15. , 14. ,
       13. , 12. , 11. , 10. ,  9. ,  8. ,  7. ,  6. ,  5. ,  4.3])

In [33]:
vpu.set_gap_speed(1.0)

True

In [38]:
data = dict()
data['gap'] = list()
data['timestamps'] = list()
data['tunex_bbb'] = list()
data['tuney_bbb'] = list()
data['tunex_dev'] = list()
data['tuney_dev'] = list()
# data['CAX_images'] = list()
t0 = time.time()
for i, gap in enumerate(gaps):

    # Move gap
    success = move_vpu_gap_robust(vpu, gap=gap, timeout=timeout, verbose=True)
    
    # Verify if undulator moved
    if not success:
        break

    # Wait orbit distortion to become stationary
    time.sleep(1)
    
    # Record orbit and other info
    data['gap'].append(gap)
    data['timestamps'].append(time.time())
    data['beamline'] = IDSearch.conv_idname_2_beamline(vpu.devname)

    # Record CAX img params and tune
    result = get_tune_data(bbb_v=bbb_v, bbb_h=bbb_h, tune=tune, nr_acq=20, acq_sleep=1)
    tunex, tuney, tunex_dev, tuney_dev = result
    data['tunex_bbb'].append(tunex)
    data['tuney_bbb'].append(tuney)
    data['tunex_dev'].append(tunex_dev)
    data['tuney_dev'].append(tuney_dev)

print(f'Done! ETA: {time.time()-t0:.3f}s')
save(data, 'VPU_{}_tune_variation_iter1.pickle'.format(IDSearch.conv_idname_2_beamline(vpu.devname)))

Gap-RB 24.000 mm
Undulator is moving...
Gap 24.000 mm reached.
Movimentation done!

Gap-RB 23.000 mm
Undulator is moving...
Gap 23.000 mm reached.
Movimentation done!

Gap-RB 22.000 mm
Undulator is moving...
Gap 22.000 mm reached.
Movimentation done!

Gap-RB 21.000 mm
Undulator is moving...
Gap 21.000 mm reached.
Movimentation done!

Gap-RB 20.000 mm
Undulator is moving...
Gap 20.000 mm reached.
Movimentation done!

Gap-RB 19.000 mm
Undulator is moving...
Gap 19.000 mm reached.
Movimentation done!

Gap-RB 18.000 mm
Undulator is moving...
Gap 18.000 mm reached.
Movimentation done!

Gap-RB 17.000 mm
Undulator is moving...
Gap 17.000 mm reached.
Movimentation done!

Gap-RB 16.000 mm
Undulator is moving...
Gap 16.000 mm reached.
Movimentation done!

Gap-RB 15.000 mm
Undulator is moving...
Gap 15.000 mm reached.
Movimentation done!

Gap-RB 14.000 mm
Undulator is moving...
Gap 14.000 mm reached.
Movimentation done!

Gap-RB 13.000 mm
Undulator is moving...
Gap 13.000 mm reached.
Movimentation